In [1]:
import numpy as np
import pandas as pd
import random
import math

In [2]:
raw_rating_data = pd.read_csv('ratings.csv')
all_users_id = list(set(list(raw_rating_data['userId'])))
all_users_id.sort()
last_user_id = max(all_users_id)
user_movie_df = pd.read_csv('test1.csv')
raw_movies_data = pd.read_csv('movies.csv')
all_movies_id = set(list(raw_movies_data['movieId']))
trans = list(all_movies_id)
trans.sort()

In [3]:
def test_users_select(num, num2, num3):
    test_users=[]                     # 显然num3要小于num2
    test_users_movies=[]
    test_users_movies_split=[]
    index = random.sample(range(1,last_user_id+1),last_user_id)
    k=0
    for i in index:
        sub_user_rating_num = len(list(raw_rating_data['userId'][raw_rating_data.userId==i]))
        sub_user_rating_movie = list(raw_rating_data['movieId'][raw_rating_data.userId==i])
        if sub_user_rating_num>=num2:
            if k>=num:
                break
            k=k+1
            split_index = random.sample(range(0,sub_user_rating_num),num2)
            split_index = [split_index[:num3],split_index[num3:]]
            test_users.append(i)
            test_users_movies.append(sub_user_rating_movie)
            test_users_movies_split.append(split_index)
    return test_users, test_users_movies, test_users_movies_split

In [4]:
test_users, test_users_movies, test_users_movies_split=test_users_select(20, 140, 100)

In [5]:
def I(v):
    v=list(v)
    if (len(v)==0):
        return 0
    index1=[i for i,x in enumerate(v) if x==1]
    index0=[i for i,x in enumerate(v) if x==0]
    p1=len(index1)/len(v)
    p0=len(index0)/len(v)
    if p1==1 or p0==1:
        return 0
    return -p1*math.log(p1, 2)-p0*math.log(p0, 2)

In [6]:
def pure_predict(userId, train_moiveId, target_item, num):
    list_all = [i for i in range(0,2*userId-2)]+[i for i in range(2*userId,2*last_user_id)]
    train_columns = [str(trans.index(i)+1) for i in train_movieId]
    sub_df = user_movie_df[train_columns]
    observe_df = user_movie_df[[str(trans.index(target_item)+1)]]
    true_rating = observe_df.loc[[2*userId-2]]
    true_rating = true_rating.values[0][0]
    features_all = sub_df.loc[list_all]
    userlike_df = sub_df.loc[[2*userId-2]]
    userlike_data = userlike_df.values[0]
    return (sum(userlike_data)/len(userlike_data)>0.5), true_rating

In [7]:
def trans_model_predict(userId, train_moiveId, target_item, num):
    features_all = all_users_id[:userId]+all_users_id[(userId+1):]
    train_columns = [str(trans.index(i)+1) for i in train_movieId]
    sub_df = user_movie_df[train_columns]
    #print(sub_df)
    list_all = [i for i in range(0,2*userId-2)]+[i for i in range(2*userId,2*last_user_id)]
    features_all = sub_df.loc[list_all]
    userlike_df = sub_df.loc[[2*userId-2]]
    
    #print(features_all,userlike_df)
    observe_df = user_movie_df[[str(trans.index(target_item)+1)]]
    true_rating = observe_df.loc[[2*userId-2]]
    
    true_rating = true_rating.values[0][0]
    observe_df = observe_df.loc[list_all]
    #rint(true_rating,observe_df)
    #print(observe_df)
    def entropy(v2):
        v1=userlike_df.values
        #print(v1)
        v1=v1[0]
        #print(v1)
        index1=[i for i,x in enumerate(v2) if x==1]
        index0=[i for i,x in enumerate(v2) if x==0]
        gain = I(v1)-len(index1)/len(v2)*I(v1[index1])-len(index0)/len(v2)*I(v1[index0])
        return gain
    all_info_gain = list(map(entropy, features_all.values))
    best_num = np.argsort(-np.array(all_info_gain))[:num]
    #rint(best_num)
    list_all2 = [list_all[i] for i in best_num]
    best_feature = features_all.loc[list_all2]
    #####(best_feature)
    #####(userlike_df)
    observe_feature = observe_df.loc[list_all2]
    #####(observe_feature)
    #print(userlike_df, observe_feature, best_feature)
    def bayes_classfier(user_data, observe_data, feature_data):
        user_data=user_data[0]
        index1=[i for i,x in enumerate(user_data) if x==1]
        index0=[i for i,x in enumerate(user_data) if x==0]
        def cal_condp1(patial_feature, ob):
            if ob==1:
                condp=(sum(patial_feature[index1])+1)/(len(index1)+2)
            else:
                condp=(len(index1)-sum(patial_feature[index1])+1)/(len(index1)+2)
            return condp
        def cal_condp0(patial_feature, ob):
            if ob==1:
                condp=(sum(patial_feature[index0])+1)/(len(index0)+2)
            else:
                condp=(len(index0)-sum(patial_feature[index0])+1)/(len(index0)+2)
            return condp
        all_condp1 = [cal_condp1(feature_data[i],observe_data[i]) for i in range(len(feature_data))]
        all_condp0 = [cal_condp0(feature_data[i],observe_data[i]) for i in range(len(feature_data))]
        all_logp1 = [math.log(i,2) for i in all_condp1]
        all_logp0= [math.log(i,2) for i in all_condp0]
        if (len(user_data)-sum(user_data))==0:
            return True, true_rating
        if sum(user_data)==0:
            return False, true_rating
        condp1=math.log(sum(user_data)/len(user_data),2)+sum(all_logp1)
        
        #####(sum(user_data))
        condp0=math.log((len(user_data)-sum(user_data)+0.0001)/len(user_data),2)+sum(all_logp0)
        #####(condp1,condp0)
        return (condp1>condp0), true_rating
    return bayes_classfier(userlike_df.values, observe_feature.values, best_feature.values)

In [8]:
def sparse_model_predict(userId, train_moiveId, target_item, num):
    users_all = all_users_id[:userId]+all_users_id[(userId+1):]
    train_columns = [str(trans.index(i)+1) for i in train_movieId]
    sub_df = user_movie_df[train_columns]
    #print(sub_df)
    list_all = [i for i in range(0,2*userId-2)]+[i for i in range(2*userId,2*last_user_id)]
    features_all = sub_df.loc[list_all]
    userlike_df = sub_df.loc[[2*userId-2]]
    
    #print(features_all,userlike_df)
    observe_df = user_movie_df[[str(trans.index(target_item)+1)]]
    true_rating = observe_df.loc[[2*userId-2]]
    
    true_rating = true_rating.values[0][0]
    observe_df = observe_df.loc[list_all]
    #rint(true_rating,observe_df)
    #print(observe_df)
    def entropy(v2):
        S=userlike_df.values
        v1=v2[0]
        v0=v2[1]
        v_common = list(np.array(v1)+np.array(v0))
        if sum(v_common)==0:
            return 0, []
        index_common = [i for i,x in enumerate(v_common) if x==1]
        S=S[0]
        index1=[i for i,x in enumerate(v1) if x==1]
        index0=[i for i,x in enumerate(v1) if x==0]
        gain1 = I(S)-len(index1)/len(v1)*I(S[index1])-len(index0)/len(v1)*I(S[index0])
        index1=[i for i,x in enumerate(v0) if x==1]
        index0=[i for i,x in enumerate(v0) if x==0]
        gain0 = I(S)-len(index1)/len(v0)*I(S[index1])-len(index0)/len(v0)*I(S[index0])
        index1=[i for i,x in enumerate(v1[index_common]) if x==1]
        index0=[i for i,x in enumerate(v1[index_common]) if x==0]
        S=S[index_common]
        #print(S)
        gain_common = I(S)-len(index1)/len(index_common)*I(S[index1])-len(index0)/len(index_common)*I(S[index0])
        return gain0*gain1*gain_common, index_common
    
    feature_single = features_all.values
    observe_single = observe_df.values
    feature_double = [[feature_single[2*i], feature_single[2*i+1]] for i in range(len(users_all))]
    observe_double = [[observe_single[2*i], observe_single[2*i+1]] for i in range(len(users_all))]
    #all_info_gain = list(map(entropy, feature_double))
    all_info_gain=[]
    all_info_index=[]
    for i in range(len(users_all)):
        a,b = entropy(feature_double[i])
        all_info_gain.append(a)
        all_info_index.append(b)
    best_num = np.argsort(-np.array(all_info_gain))
    #rint(best_num)
    list_all2 = [list_all[2*i] for i in best_num]
    index_data = [all_info_index[i] for i in best_num]
    #best_feature = features_all.loc[list_all2]
    best_feature = [feature_double[i] for i in best_num]
    #print(best_feature)
    #print(userlike_df)
    #observe_feature = observe_df.loc[list_all2]
    observe_feature = [observe_double[i] for i in best_num]
    k=0
    j=0
    best_feature2=[]
    observe_feature2=[]
    for k in range(len(observe_feature)):
        if j>=num:
            break
        if sum(observe_feature[k])!=0:
            j=j+1
            best_feature2.append(best_feature[k])
            observe_feature2.append(observe_feature[k])
    #print(observe_feature)
    #print(userlike_df, observe_feature, best_feature)
    def bayes_classfier(user_data, observe_data, feature_data, index_data):
        user_data=user_data[0]
        def cal_condp1(patial_feature, ob, j):
            print(index_data[j])
            if sum(ob)==0:
                return 1
            patial_feature=patial_feature[ob.index(1)]
            print(ob.index(1))
            user_data1=user_data[index_data[j]]
            patial_feature=patial_feature[index_data[j]]
            print(user_data1, patial_feature)
            index1=[i for i,x in enumerate(user_data1) if x==1]
            index0=[i for i,x in enumerate(user_data1) if x==0]
            condp=(sum(patial_feature[index1])+1)/(len(index1)+2)
            return condp
        def cal_condp0(patial_feature, ob, j):
            if sum(ob)==0:
                return 1
            patial_feature=patial_feature[ob.index(1)]
            user_data0=user_data[index_data[j]]
            patial_feature=patial_feature[index_data[j]]
            index1=[i for i,x in enumerate(user_data0) if x==1]
            index0=[i for i,x in enumerate(user_data0) if x==0]
            condp=(sum(patial_feature[index0])+1)/(len(index0)+2)
            return condp
        all_condp1 = [cal_condp1(feature_data[i],observe_data[i],i) for i in range(len(feature_data))]
        all_condp0 = [cal_condp0(feature_data[i],observe_data[i],i) for i in range(len(feature_data))]
        all_logp1 = [math.log(i,2) for i in all_condp1]
        all_logp0= [math.log(i,2) for i in all_condp0]
        condp1=math.log(sum(user_data)/len(user_data),2)+sum(all_logp1)
        print(all_condp1,all_condp0)
        print(sum(user_data))
        condp0=math.log((len(user_data)-sum(user_data))/len(user_data),2)+sum(all_logp0)
        #print(condp1,condp0)
        return (condp1>condp0), true_rating
    return bayes_classfier(userlike_df.values, observe_feature2, best_feature2, index_data)

In [9]:
iteration = [1,10,20,40,100,200,400,800,1200]
for num in iteration:
    all_wrong=0
    for j in range(20):
        for i in range(40):
            userId = test_users[j]
            train_movieId = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]
            target_item = [test_users_movies[j][i] for i in test_users_movies_split[j][1]][i]
            a,b = trans_model_predict(userId, train_movieId, target_item, num)
            if (a-b)!=0:
                all_wrong=all_wrong+1
    print(all_wrong)

241
226
247
239
254
251
258
242
226


In [67]:
iteration = [1,10,20,40,100,200,400,800,1200]
for num in iteration[:2]:
    all_wrong=0
    for j in range(20):
        for i in range(40):
            userId = test_users[j]
            train_movieId = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]
            target_item = [test_users_movies[j][i] for i in test_users_movies_split[j][1]][i]
            a,b = pure_predict(userId, train_movieId, target_item, num)
            if (a-b)!=0:
                all_wrong=all_wrong+1
    print(all_wrong)

229
229


In [59]:
iteration = [10,20,30,40,50,60,70]
for num in iteration:
    all_wrong=0
    for j in range(20):
        all_movie = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]+[test_users_movies[j][i] for i in test_users_movies_split[j][1]]
        for i in range(90-num):
            userId = test_users[j]
            train_movieId = all_movie[:num]
            target_item = all_movie[num+i]
            a,b = trans_model_predict(userId, train_movieId, target_item, 1200)
            if (a-b)!=0:
                all_wrong=all_wrong+1
    print(all_wrong)

392
348
299
252
221
171
112


In [60]:
iteration = [10,20,30,40,50,60,70]
for num in iteration:
    all_wrong=0
    for j in range(20):
        all_movie = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]+[test_users_movies[j][i] for i in test_users_movies_split[j][1]]
        for i in range(90-num):
            userId = test_users[j]
            train_movieId = all_movie[:num]
            target_item = all_movie[num+i]
            a,b = pure_predict(userId, train_movieId, target_item, 1200)
            if (a-b)!=0:
                all_wrong=all_wrong+1
    print(all_wrong)

392
344
307
237
194
147
98


In [30]:
j=1
i=0
num=5
for i in range(4,5):
    userId = test_users[j]
    train_movieId = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]
    target_item = [test_users_movies[j][i] for i in test_users_movies_split[j][1]][i]
    a,b = sparse_model_predict(userId, train_movieId, target_item, num)
    print(a,b)

[2, 3, 4, 5, 15, 22, 24, 33, 34, 46]
1
[1 0 1 1 1 0 1 1 1 1] [0 0 0 0 0 0 0 0 0 0]
[4, 6, 12, 16, 23, 25, 26, 30, 31, 35, 41, 43, 45, 48]
0
[1 0 0 0 1 1 1 1 1 1 1 0 0 1] [1 0 0 0 1 0 0 0 1 0 1 1 1 0]
[14, 23, 34, 37, 40, 41]
0
[1 1 1 0 0 1] [0 0 0 0 0 0]
[0, 1, 3, 4, 6, 8, 12, 13, 14, 16, 18, 19, 20, 23, 25, 26, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 48, 49]
0
[0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1, 27, 34, 35, 40]
1
[0 1 1 1 0] [0 0 0 0 0]
[0.1, 0.45454545454545453, 0.16666666666666666, 0.1, 0.2] [0.25, 0.42857142857142855, 0.25, 0.06666666666666667, 0.25]
33
False 0


In [34]:
num=2
all_wrong=0
for j in range(4,5):
    for i in range(30):
        userId = test_users[j]
        train_movieId = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]
        target_item = [test_users_movies[j][i] for i in test_users_movies_split[j][1]][i]
        a,b = sparse_model_predict(userId, train_movieId, target_item, num)
        if (a-b)!=0:
            all_wrong=all_wrong+1
print(all_wrong)

[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
1
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [0 0 0 0 0 1 0 0 0 1 0 1 0 0]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
1
[1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
[0.16666666666666666, 0.058823529411764705] [0.5, 0.2727272727272727]
29
[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
0
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [1 1 1 1 0 0 1 1 0 1 1 0 1 1]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
0
[1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0] [0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0]
[0.8333333333333334, 0.4117647058823529] [0.3333333333333333, 0.09090909090909091]
29
[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
0
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [0 0 1 1 0 0 0 1 0 1 1 0 1 1]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
1
[1 1 1 1 0 1 0 1 

[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
0
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [1 0 1 1 0 0 0 1 0 1 1 0 1 1]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
1
[1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0] [0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0]
[0.6666666666666666, 0.29411764705882354] [0.3333333333333333, 0.09090909090909091]
29
[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
0
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [0 0 1 1 0 0 0 1 0 1 0 0 1 0]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
0
[1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0] [1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1]
[0.5, 0.6470588235294118] [0.16666666666666666, 0.2727272727272727]
29
[1, 2, 3, 4, 7, 12, 13, 14, 16, 19, 22, 29, 35, 41]
0
[1 1 1 1 0 0 1 1 1 1 1 0 1 0] [0 0 0 0 0 0 0 0 0 1 1 0 0 0]
[1, 2, 3, 4, 7, 11, 12, 13, 14, 15, 19, 21, 22, 24, 28, 29, 31, 32, 35, 36, 41, 45, 47, 49]
1
[1 1 1 1 0 1 0 1 1

In [42]:
num=100
all_wrong=0
for j in range(20):
    for i in range(30):
        userId = test_users[j]
        train_movieId = [test_users_movies[j][i] for i in test_users_movies_split[j][0]]
        target_item = [test_users_movies[j][i] for i in test_users_movies_split[j][1]][i]
        a,b = pure_predict(userId, train_movieId, target_item, num)
        if (a-b)!=0:
            all_wrong=all_wrong+1
print(all_wrong)

176
